# Marathi2Vec Language Modeling

Thanks to NirantK and cstorm125 for the works Hindi2Vec and Thai2Vec, both of which helped me make this notebook. 

The goal of this notebook is to train Marathi word embeddings using the fast.ai version of AWD LSTM with dropouts, the data was fulled from Wikipedia.

A perplexity of was achieved, there has been no comparable research work in Marathi Language at the point of writing. (2nd October, 2018)


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import dill as pickle
import json 
import re

from IPython.display import Image
from IPython.core.display import HTML

import torchtext
from torchtext import vocab
from torchtext import data as d
from torchtext.datasets import language_modeling

In [ ]:
from fastai.learner import *
from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

In [13]:

!cd Data


Data  LICENSE  Marathi2Vec.ipynb  README.md


 - Download or clone WikiExtractor from github
 - To run on windows look at [this](https://github.com/attardi/wikiextractor/issues/89#issuecomment-272062219).
 
 Run the following command :
 
 ```python WikiExtractor.py mrwiki-latest-pages-articles.xml -o extract -b 10M --ignored_tags abbr,b,big --discard_elements gallery,timeline,noinclude```

In [28]:
%%time
%%prun

data = 'Data/'
extract= f'{data}extract/'
train_set = f'{data}train/'
valid_set = f'{data}valid/'
models = f'{data}models/'


ext =  !ls {extract}

def cleanFile(extracted_filelist, dest):    
    cleaned_all = []
    for ext_file in extracted_filelist:
        input_file = f'{extract}{ext_file}'
        with open(input_file,'r', encoding='utf-8') as f:
            raw_txt = f.readlines()
            cleaned_doc = []
            for line in raw_txt:
                new_line = re.sub('<[^<]+?>', '', line)
                new_line = re.sub('__[^<]+?__', '', new_line) 
                new_line = new_line.strip()
                if new_line != '':
                    cleaned_doc.append(new_line)

            new_doc = "\n".join(cleaned_doc)
            cleaned_all.append(new_doc)
            with open(f"{dest}{ext_file}.txt", "w", encoding='utf-8') as text_file:
                text_file.write(new_doc)
    return cleaned_all

cleaned_all = cleanFile(ext, train_set)



 Wall time: 10.8 s


## Create Validation Set

In [29]:
import random
random.seed(782)

training_files = !ls {train_set}
validation_files = !ls {valid_set} 
split = int(0.2 * len(training_files)) #Doing a 80-20 split
random.shuffle(training_files)
validation_files = training_files[:split]
training_files = training_files[split:]

print(validation_files)
print(training_files)

['wiki_00.txt', 'wiki_06.txt']
['wiki_09.txt', 'wiki_07.txt', 'wiki_03.txt', 'wiki_02.txt', 'wiki_08.txt', 'wiki_05.txt', 'wiki_04.txt', 'wiki_01.txt']


In [33]:
#import shutil, os
#for root, dirs, files in os.walk(train_set):
#     for file in files:
#         if file.endswith(".txt") & (file in validation_files):
#             shutil.move(os.path.join(root, file),valid_set)
trn_files = !ls {train_set}
val_files = !ls {valid_set}
print(trn_files), print(val_files), print(len(trn_files)), print(len(val_files))

['wiki_01.txt', 'wiki_02.txt', 'wiki_03.txt', 'wiki_04.txt', 'wiki_05.txt', 'wiki_07.txt', 'wiki_08.txt', 'wiki_09.txt']
['wiki_00.txt', 'wiki_06.txt']
8
2


(None, None, None, None)

# Text Processing

There aren't any known Marathi Stemmers available in Python, So i've written these porting taking inspiration from Spacy's hindi tokenizer and stemmer, Since Marathi and Hindi Inherently come from the same script i.e Devanagri.

In [34]:
 from cltk.tokenize.sentence import TokenizeSentence

In [35]:
def word_tokenize(document):
    tokenizer = TokenizeSentence('marathi')
    return tokenizer.tokenize(document)
    
def docs_tokenize(documents_as_lists):   
    for document in documents_as_lists:
        tokens = word_tokenize(document)
        tokens_list.extend(tokens)
    
    return tokens_list

In [36]:
%%time
tokens_filename = "tokens_list.txt"
tokens_list = []

#TODO refactor from try except blocks to if else with if statement checking if file exists using Pathlib

try:
    print(f'Reading from {tokens_filename}')
    with open(tokens_filename, "r") as f:
         tokens_list = json.load(f)
    
except FileNotFoundError:
    print(f'FileNotFound. Trying to tokenize from cleaned_all')
    tokens_list = docs_tokenize(cleaned_all)
    
    with open('tokens_list.txt', 'w') as outfile:
        json.dump(tokens_list, outfile)

print(f'Found {len(tokens_list)} tokens')

Reading from tokens_list.txt
Found 6307733 tokens
Wall time: 8.63 s


In [37]:
assert torch.cuda.is_available()
assert torch.backends.cudnn.enabled

In [38]:
TEXT = d.Field(lower=True, tokenize=word_tokenize)
#batch size
bs=16
#backprop through time
bptt=70
test_set = f'{data}test/'

In [39]:
FILES = dict(train=f'{train_set}', validation=f'{valid_set}', test=f'{test_set}')

In [43]:
%%time
md = LanguageModelData.from_text_files('', TEXT, **FILES, bs=bs, bptt=bptt, min_freq=50)

G:\conda\envs\fastai\lib\site-packages\torch\cuda\__init__.py:116: UserWarning: 
    Found GPU0 GeForce 940MX which is of cuda capability 5.0.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


RuntimeError: cuda runtime error (48) : no kernel image is available for execution on the device at C:/Anaconda2/conda-bld/pytorch_1519501749874/work/torch/lib/THC/THCTensorCopy.cu:204